In [ ]:
options(dplyr.summarise.inform = FALSE)
library(tidyverse)
library(data.table)
library(survival)
library(gridExtra)

In [ ]:
source(paste0(dirname(dirname(dirname(getwd()))),'/map.r'))
source(paste0(dirname(dirname(dirname(getwd()))),'/stats.r'))

#### 0 - Run analyses

In [ ]:
system.time(go <- readRDS(paste0(SHARE_DIR, "ready_ex.Rds")))

In [ ]:
df <- go$data_ready
features <- go$features

#### 1 - Run Survival Analyses

- Survival, test everything

In [ ]:
system.time(os_results <- scanner("Surv(Y_os_days, Y_os_event)", features, "+ location + clin_age + clin_sex", "df", "coxph"))

In [ ]:
get_stats( y = "Surv(Y_os_days, Y_os_event)", 
                        x = i, 
                        covariate = " + clin_primaryTumorLocation2 + clin_age + clin_sex", 
                        data = "df", 
                        model = "coxph")

In [ ]:
results <- data.frame()
system.time(
for( i in features){
    results <- 
      rbind(results, 
            get_stats2( y = "Surv(Y_os_days, Y_os_event)", 
                        x = i, 
                        covariate = " + clin_primaryTumorLocation2 + clin_age + clin_sex", 
                        data = "df", 
                        model = "coxph"))
})

In [ ]:
fwrite(results %>% arrange(pval), paste0(UTIL_DIR, "zscores_tmp.csv"))

#### 2 - Relations with top Z-scores

- Linear model with proliferation

In [ ]:
results <- fread(paste0(UTIL_DIR, "zscores_tmp.csv"))

In [ ]:
top_zscores <- results %>% filter(grepl("zscore", x)) %>% arrange(pval) %>% head(20) %>% pull(x)

In [ ]:
lms <- data.frame()
for(j in top_zscores){
  print(j); flush.console()
  for( i in features){
    lms <- rbind(lms, 
                 get_stats2( y = j, 
                 x = i, 
                 covariate = " + clin_primaryTumorLocation2 + clin_age + clin_sex", 
                 data = "df", 
                 model = "lm"))
}}

In [ ]:
lms <- lms %>% mutate(pval_by = p.adjust(pval, method = "BY")) 

In [ ]:
fwrite(lms, paste0(UTIL_DIR, "zscores_tmp2.csv"))